In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

df = pd.read_csv('SpotifyFeatures.csv')

df.loc[df.loc[:,'genre']=='Rap','genre'] = 'Hip-Hop'
df = df.loc[df.loc[:, 'popularity']>= 60, :]

df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
135,R&B,Mary J. Blige,Be Without You - Kendu Mix,2YegxR5As7BeQuVp2U6pek,65,0.0830,0.724,246333,0.689,0.000000,D,0.3040,-5.922,Minor,0.1350,146.496,4/4,0.6930
136,R&B,Rihanna,Desperado,6KFaHC9G178beAp7P0Vi5S,63,0.3230,0.685,186467,0.610,0.000000,C,0.1020,-5.221,Minor,0.0439,94.384,3/4,0.3230
137,R&B,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,6muW8cSjJ3rusKJ0vH5olw,62,0.0675,0.762,199520,0.520,0.000004,F,0.1140,-5.237,Minor,0.0959,75.047,4/4,0.0862
138,R&B,Surfaces,Heaven Falls / Fall on Me,7yHqOZfsXYlicyoMt62yC6,61,0.3600,0.563,240597,0.366,0.002430,B,0.0955,-6.896,Minor,0.1210,85.352,4/4,0.7680
139,R&B,Olivia O'Brien,Love Myself,4XzgjxGKqULifVf7mnDIQK,68,0.5960,0.653,213947,0.621,0.000000,B,0.0811,-5.721,Minor,0.0409,100.006,4/4,0.4660


In [2]:
from sklearn.decomposition import PCA

num_components = 3
cols = ['artist_name','track_name','track_id']

def prepro(df, cols, num_components):

    dummies = pd.get_dummies(df.loc[:, ~df.columns.isin(cols)])
    
    merged = pd.merge(df.loc[:, cols], dummies, left_index=True, right_index=True)
    
    merged = merged.groupby(cols).max()
    
    genres = merged.loc[:, [i for i in merged.columns if i.startswith('genre')]]
            
    x = merged.iloc[:, 11:]

    pca = PCA(n_components=num_components)
    x_pca = pca.fit_transform(x)
    
    merged = merged.iloc[:, :11]
    
    for i in range(num_components):
        merged.loc[:, f'pca_{i+1}'] = x_pca[:, i]
        
    for col in merged.columns:
        mn, mx = min(merged.loc[:, col]), max(merged.loc[:, col])
        
        merged.loc[:, col] -= mn
        merged.loc[:, col] /= (mx - mn)

    
    return merged, genres
    
    
tmp, genres = prepro(df, cols, num_components)

tmp = tmp.sort_values(['popularity'], ascending=False)

for i, col in enumerate(tmp.columns):
    print(i, col)
    
tmp.head()

0 popularity
1 acousticness
2 danceability
3 duration_ms
4 energy
5 instrumentalness
6 liveness
7 loudness
8 speechiness
9 tempo
10 valence
11 pca_1
12 pca_2
13 pca_3


popularity  \
artist_name   track_name                               track_id                             
Ariana Grande 7 rings                                  14msK75pk3pA33pzPVNtBF       1.000   
              break up with your girlfriend, i'm bored 4kV4N9D1iKVxx1KLvtTpjS       0.975   
Post Malone   Wow.                                     6MWtB6iiXyIwun0YzU6DFP       0.975   
Daddy Yankee  Con Calma                                5w9c2J52mkdntKOmRLeM2m       0.950   
Marshmello    Happier                                  2dpaYNEQHiRxtZbfNsse99       0.925   

                                                                               acousticness  \
artist_name   track_name                               track_id                               
Ariana Grande 7 rings                                  14msK75pk3pA33pzPVNtBF      0.580320   
              break up with your girlfriend, i'm bored 4kV4N9D1iKVxx1KLvtTpjS      0.042267   
Post Malone   Wow.                                     6MWtB6iiXyIwun0YzU6DFP      0.163653   
Daddy Yankee  Con Calma                                5w9c2J52mkdntKOmRLeM2m      0.110440   
Marshmello    Happier                                  2dpaYNEQHiRxtZbfNsse99      0.191765   

                                                                               danceability  \
artist_name   track_name                               track_id                               
Ariana Grande 7 rings                                  14msK75pk3pA33pzPVNtBF      0.723890   
              break up with your girlfriend, i'm bored 4kV4N9D1iKVxx1KLvtTpjS      0.724981   
Post Malone   Wow.                                     6MWtB6iiXyIwun0YzU6DFP      0.841755   
Daddy Yankee  Con Calma                                5w9c2J52mkdntKOmRLeM2m      0.736986   
Marshmello    Happier                                  2dpaYNEQHiRxtZbfNsse99      0.682418   

                                                                               duration_ms  \
artist_name   track_name                               track_id                              
Ariana Grande 7 rings                                  14msK75pk3pA33pzPVNtBF     0.147918   
              break up with your girlfriend, i'm bored 4kV4N9D1iKVxx1KLvtTpjS     0.159794   
Post Malone   Wow.                                     6MWtB6iiXyIwun0YzU6DFP     0.118611   
Daddy Yankee  Con Calma                                5w9c2J52mkdntKOmRLeM2m     0.162599   
Marshmello    Happier                                  2dpaYNEQHiRxtZbfNsse99     0.183797   

                                                                                 energy  \
artist_name   track_name                               track_id                           
Ariana Grande 7 rings                                  14msK75pk3pA33pzPVNtBF  0.320342   
              break up with your girlfriend, i'm bored 4kV4N9D1iKVxx1KLvtTpjS  0.553912   
Post Malone   Wow.                                     6MWtB6iiXyIwun0YzU6DFP  0.538875   
Daddy Yankee  Con Calma                                5w9c2J52mkdntKOmRLeM2m  0.860660   
Marshmello    Happier                                  2dpaYNEQHiRxtZbfNsse99  0.792494   

                                                                               instrumentalness  \
artist_name   track_name                               track_id                                   
Ariana Grande 7 rings                                  14msK75pk3pA33pzPVNtBF          0.000000   
              break up with your girlfriend, i'm bored 4kV4N9D1iKVxx1KLvtTpjS          0.000000   
Post Malone   Wow.                                     6MWtB6iiXyIwun0YzU6DFP          0.000002   
Daddy Yankee  Con Calma                                5w9c2J52mkdntKOmRLeM2m          0.000002   
Marshmello    Happier                                  2dpaYNEQHiRxtZbfNsse99          0.000000   

                                                                               liveness  \
artist_name  

In [12]:
genre_dict = {}

for i in genres.index:
    genre_dict[i[2]] = []
    
    ar = genres.loc[i,:]
    
    for j in ar.index:        
        if ar[j] == 1:
            genre_dict[i[2]].append(j.split('_')[1])


In [13]:
from sklearn.cluster import KMeans

num_clust = 15

kmeans = KMeans(n_clusters = num_clust, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(tmp)

tmp.loc[:, 'cluster'] = y_kmeans

C:\Users\dimaz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2819: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


In [14]:
col_nums = {}

for i, c in enumerate(tmp.columns[:-1]):
    col_nums[c] = i
    
col_nums

{'popularity': 0,
 'acousticness': 1,
 'danceability': 2,
 'duration_ms': 3,
 'energy': 4,
 'instrumentalness': 5,
 'liveness': 6,
 'loudness': 7,
 'speechiness': 8,
 'tempo': 9,
 'valence': 10,
 'pca_1': 11,
 'pca_2': 12,
 'pca_3': 13}

In [15]:
import json
    
dropdownDB = {}

for i, p in zip(tmp.index, tmp.loc[:,'popularity']):
    dropdownDB[i[2]] = [f'{i[0]} - {i[1]}',p]
    
with open('dropdownDB.json', 'w') as f:
    json.dump(dropdownDB, f)

In [16]:
prefix = '6i'

f = open('dropdownDB.json',) 

dropdownDB = json.load(f) 

for k, v in dropdownDB.items():
    if len(v[0].lower().split(prefix)) > 1:
        print(k, v)

1wJRveJZLSb1rjhnUHQiv6 ['A Boogie Wit da Hoodie - Swervin (feat. 6ix9ine)', 0.775]
6Gy7rXB6Ku5vIWC7WGWsl3 ['6ix9ine - KIKA (feat. Tory Lanez)', 0.7]
4KkUjFkcQvGvWekl8RATDn ['6ix9ine - BEBE (feat. Anuel Aa)', 0.7]
0LQ1J0BCzA2qnAgp9MkkNE ['6ix9ine - MALA (feat. Anuel Aa)', 0.675]
4QIyEq7TcTdAcxjI2AFlAK ['6ix9ine - FEFE (feat. Nicki Minaj, Murda Beatz)', 0.65]
1oJ8EDr2pv94IvSd6ZmDgo ['6ix9ine - MAMA (feat. Nicki Minaj, Kanye West)', 0.525]
4m5Cd9kAbIxg6t9sMrQWtk ['6ix9ine - TIC TOC (feat. Lil Baby)', 0.525]
6o79O9qBP3OAkWOXajAQIS ['6ix9ine - WAKA (feat. A Boogie Wit da Hoodie)', 0.5]
6k2iq1yULhXILe0i8xQqPF ['6ix9ine - STOOPID (feat. Bobby Shmurda)', 0.475]
5KhFaq45chTw8RGfWo8T8J ['6ix9ine - BILLY', 0.475]
6wJrqTE6c4NtfDqAcSRcBf ['6ix9ine - TATI (feat. DJ SpinKing)', 0.45]
27oFpFktCq9JIXjJQRuv7a ['6ix9ine - GOTTI', 0.425]
4HbiCvH1R7mVOJ7KY7JQBD ['6ix9ine - GUMMO', 0.4]
0EwS8XJQspgrj1zSUXhFkl ['6ix9ine - KANGA (feat. Kanye West)', 0.35]
14PlDNjNh3pXyHXzkhX8n5 ['6ix9ine - KEKE', 0.325]
0tYHq

In [17]:
out = {}

for i in list(col_nums.keys()) + ['cluster']:
    out[i] = list(tmp.loc[:, i].values)

out['track_id'] = [i[2] for i in tmp.index]
out['cluster'] = [int(i) for i in out['cluster']]

with open('plotData.json', 'w') as f:
    json.dump(out, f)

In [18]:
# cosine similarity

from scipy import spatial
import matplotlib.patches as mpatches
from matplotlib import transforms

inds = {
    'index': 20
}

for i in inds.values():
    print(f'{tmp.iloc[i, :].name[0]} - {tmp.iloc[i, :].name[1]}')
    
def similar(inds, tmp, genres):
    
    target_ind = inds['index']
    target_vals = tmp.iloc[target_ind, :]
    
    genres = genres.reindex(tmp.index)
    
    tmp_genres = genres.iloc[target_ind, :]
    
    genres.loc[:, 'cluster'] = tmp.loc[:, 'cluster']

    clust = tmp['cluster'].iloc[target_ind]
    tmp = tmp.loc[tmp.loc[:, 'cluster'] == clust, :]
    
    tmp.loc[:, 'index'] = 0
    
    print(len(tmp))

    for i in range(len(tmp)):
        x = spatial.distance.cosine(target_vals, target_vals) 

#     print('most similar songs:')
#     head = tmp.sort_values(['index']).head(10)
    
#     for i in head.index:
#         print(f'    {i[0]} - {i[1]}')
                
#     genres = genres.loc[genres.loc[:,'cluster']==clust,:].groupby(['cluster']).sum()
    
#     genres = genres.T
    
#     genres.loc[:, 'target'] = tmp_genres.T
    
#     genres.columns = ['cluster', 'target']
    
#     genres = genres.sort_values(['cluster'], ascending = True)
    
#     genres.loc[:, 'cluster'] /= genres.loc[:, 'cluster'].sum()
    
#     print(genres)

    
similar(inds, tmp, genres)

Travis Scott - SICKO MODE
1293


C:\Users\dimaz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2845: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\Users\dimaz\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\dimaz\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
# cosine simil

simil_dict = {}

for i in tmp.index:
    simil_dict[i[2]] = [
        f'{i[0]} - {i[1]}',
        list(tmp.loc[i,:][:-1]),
        list(tmp.loc[i,:])[-1],
        genre_dict[i[2]]
    ]
    
with open('similarityDb.json', 'w') as f:
    json.dump(simil_dict, f)

In [20]:
from scipy import spatial

target = '3CgZCQyuyxHRMWB9BTwmni'

target_vals = simil_dict[target][1]

cos_vals, track_names, index = [], [], []

for k, v in simil_dict.items():
    cos_vals.append(spatial.distance.cosine(target_vals, v[1]))
    track_names.append(v[0])
    index.append(k)

cosDf = pd.DataFrame({
    'index': index,
    'track_names': track_names,
    'cos_vals': cos_vals
}).sort_values(['cos_vals'], ascending=True)

out = {
    'most': {},
    'least': {}
}

for i in range(1,4):
    out['most'][i-1] = list(cosDf.iloc[i,:])

cosDf = cosDf.sort_values(['cos_vals'], ascending=False)

for i in range(3):
    out['least'][i] = list(cosDf.iloc[i,:])

out

{'most': {0: ['4mmkhcEm1Ljy1U9nwtsxUo',
   'Kanye West - Through The Wire',
   0.010819938125603779],
  1: ['1zTW7xRtxmprjTUDi6bAHg',
   'Jon Bellion - Woke The F*ck Up',
   0.010924536566403575],
  2: ['4gdQ8QShlC3toP9Mkc8DwB', 'Cam’ron - Oh Boy', 0.011272033590640329]},
 'least': {0: ['5Q5MFVjUWVNgb7XBerPdbQ',
   'Brian Eno - Signals - Remastered 2005',
   0.80616457921023],
  1: ['3FVy3aVs0TFzLv6FEiLqC2',
   'Max Richter - Dream 1 (before the wind blows it all away) - Pt. 1',
   0.7580002098581463],
  2: ['45Mxc03iwZ9jOZFmO5HqF0',
   'Max Richter - Dream 11 (whisper music) - Pt. 1',
   0.7412585762982616]}}